# Connecting AgentCore Runtime Agents to Elasticsearch via AgentCore Gateway

## Overview

Building AI agents that can intelligently search and retrieve information from your organization's data stores is essential for creating powerful customer support and knowledge management solutions. However, connecting AI agents to enterprise search systems like Elasticsearch while maintaining security, scalability, and standardization can be complex.

Amazon Bedrock AgentCore Gateway can help integrate any third-party vector store like Elasticsearch and others, giving you flexibility to work with your existing data infrastructure. Elasticsearch is particularly well-suited for enterprise AI applications because it combines powerful vector search capabilities with traditional full-text search, advanced filtering, and proven scalability—making it ideal for organizations that need to search across diverse, large-scale datasets.

This tutorial demonstrates how to build an AI agent that performs semantic search on Elasticsearch data through Amazon Bedrock AgentCore Gateway. You'll learn how to transform AWS Lambda functions into Model Context Protocol (MCP) compliant tools and connect them to AI agents running in AgentCore Runtime—all while maintaining enterprise-grade security through dual authentication.

### Tutorial Details

| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Intermediate / Integration                                |
| Agent type           | Customer Support RAG Agent                                |
| AgentCore components | Gateway, Identity, Runtime                                |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target type  | AWS Lambda                                                |
| Data Store           | Elasticsearch                                             |
| Inbound Auth         | Amazon Cognito (OAuth)                                    |
| Outbound Auth        | AWS IAM                                                   |
| LLM models           | Anthropic Claude Haiku  4.5                               |
| Example complexity   | Intermediate                                              |
| SDK used             | boto3, bedrock-agentcore, bedrock-agentcore-starter-toolkit |

### What You'll Learn

In this tutorial, you'll learn:
1. How to package AWS Lambda functions for use with AgentCore Gateway
2. How to configure dual authentication (inbound OAuth and outbound IAM) for secure access
3. How to build a Strands agent that interacts with Gateway tools
4. How to deploy the complete solution to AgentCore Runtime
5 . How to test end-to-end RAG functionality with Elasticsearch

### Architecture

This tutorial demonstrates a customer support agent that retrieves policy information from Elasticsearch:

<div style="text-align:left">
    <img src="images/runtime-gateway-elastic.png" width="90%"/>
</div>

**Architecture Flow:**
1. **User Request**: Client sends authenticated request to AgentCore Runtime
2. **Inbound Auth**: Cognito validates OAuth token for gateway access
3. **Agent Processing**: Strands agent determines which tools to use
4. **Gateway Invocation**: Agent calls MCP tools via Gateway
5. **Outbound Auth**: Gateway uses IAM to invoke Lambda
6. **Data Retrieval**: Lambda queries Elasticsearch and returns results
7. **Response Generation**: Agent synthesizes final response for user

## 0. Prerequisites

To execute this tutorial you will need:

**Software Requirements:**
* Python 3.10 or newer
* Jupyter notebook environment

**AWS Requirements:**
* AWS credentials configured with appropriate permissions for:
  - Amazon Bedrock (model access)
  - AWS Lambda (create and invoke functions)
  - Amazon Cognito (user pool management)
  - Amazon ECR (container registry)
  - IAM (role and policy management)
  - AWS Systems Manager Parameter Store
* Amazon Bedrock model access (Claude Haiku 4.5)

**Elasticsearch Requirements:**
* Active Elasticsearch index
* Elasticsearch credentials:
  - `ELASTIC_ENDPOINT_URL`: Your Elasticsearch endpoint
  - `ELASTIC_API_KEY`: API key for authentication
  - `ELASTIC_INDEX_NAME`: Index name

### Setting Up Your Elasticsearch Configuration

Before proceeding, configure your Elasticsearch connection details:

In [ ]:
# Replace these values with your actual Elasticsearch details
ELASTIC_ENDPOINT_URL = "Your Elasticsearch endpoint"
ELASTIC_API_KEY = "API key for authentication"
ELASTIC_INDEX_NAME = "Index name"

### Installing Required Dependencies

Let's install all required Python packages:

In [ ]:
!pip install -qUr requirements.txt

### Setting Up Environment

Let's import required libraries and configure our environment:

In [ ]:
# Standard library imports
import os
import sys
import uuid
import time
import shutil
import zipfile
import subprocess
from pathlib import Path

# AWS SDK imports
import boto3
from botocore.exceptions import ClientError

# Set AWS region
REGION = os.getenv('AWS_REGION', 'us-east-1')

# Configure utility imports
current_dir = os.getcwd() if '__file__' not in globals() else os.path.dirname(os.path.abspath(__file__))
sys.path.insert(0, current_dir)

# Import tutorial utilities
import utils
from utils_execution import put_ssm_parameter

print(f"✅ Environment configured for region: {REGION}")

### Uploading Policy Documents to Elasticsearch

Before we can build our RAG agent, we need to populate our Elasticsearch index with product policy documents. This step will bulk upload all text files from the `policies/` directory to your Elasticsearch instance.

> ⏱️ **Performance**: Bulk indexing is efficient for large document sets. This process typically takes a few seconds for dozens of documents.

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

print("📤 Bulk uploading policy documents to Elasticsearch...\n")

# Initialize Elasticsearch client
try:
    es_client = Elasticsearch(
        ELASTIC_ENDPOINT_URL,
        api_key=ELASTIC_API_KEY
    )
    
    # Test connection
    if es_client.ping():
        print("✅ Connected to Elasticsearch")
    else:
        print("❌ Failed to connect to Elasticsearch")
        raise Exception("Elasticsearch connection failed")
        
except Exception as e:
    print(f"❌ Error connecting to Elasticsearch: {e}")
    raise

# Define the policies directory
policies_dir = Path("policies")

if not policies_dir.exists():
    print(f"❌ Directory not found: {policies_dir}")
    print("💡 Please create a 'policies/' directory and add your .txt policy files.")
    raise FileNotFoundError(f"Directory {policies_dir} does not exist")

# Get all .txt files
policy_files = list(policies_dir.glob("*.txt"))

if not policy_files:
    print(f"⚠️  No .txt files found in {policies_dir}")
    print("💡 Add your policy documents as .txt files to the policies/ directory.")
else:
    print(f"📁 Found {len(policy_files)} policy document(s)\n")

# Prepare documents for bulk indexing
def generate_docs():
    """Generator function to yield documents for bulk indexing."""
    for idx, file_path in enumerate(policy_files, 1):
        try:
            # Read file content
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Create document structure
            doc = {
                "_index": ELASTIC_INDEX_NAME,
                "_id": file_path.stem,  # Use filename (without extension) as document ID
                "_source": {
                    "filename": file_path.name,
                    "attachment": {
                        "content": content
                    },
                    "document_type": "policy",
                    "indexed_at": "2024-01-01T00:00:00Z"  # You can use datetime.now().isoformat()
                }
            }
            
            print(f"   [{idx}/{len(policy_files)}] Preparing: {file_path.name}")
            yield doc
            
        except Exception as e:
            print(f"   ⚠️  Error reading {file_path.name}: {e}")
            continue

# Perform bulk indexing
try:
    print("\n⏳ Indexing documents...\n")
    
    # Execute bulk operation
    success, failed = bulk(
        es_client,
        generate_docs(),
        raise_on_error=False,
        refresh=True  # Make documents immediately searchable
    )
    
    print("\n" + "=" * 70)
    print(f"✅ Successfully indexed: {success} document(s)")
    
    if failed:
        print(f"⚠️  Failed to index: {failed} document(s)")
    
    # Verify indexing
    count_response = es_client.count(index=ELASTIC_INDEX_NAME)
    total_docs = count_response['count']
    print(f"📊 Total documents in index '{ELASTIC_INDEX_NAME}': {total_docs}")
    print("=" * 70)
    
    # Show sample document structure
    if total_docs > 0:
        print("\n📄 Sample document structure:")
        sample_doc = es_client.search(
            index=ELASTIC_INDEX_NAME,
            size=1
        )['hits']['hits'][0]
        
        print(f"   Document ID: {sample_doc['_id']}")
        print(f"   Filename: {sample_doc['_source'].get('filename')}")
        print(f"   Content preview: {sample_doc['_source']['attachment']['content'][:100]}...")
    
    print("\n🎉 Bulk upload completed successfully!\n")
    
except Exception as e:
    print(f"\n❌ Error during bulk indexing: {e}")
    import traceback
    traceback.print_exc()
    raise

finally:
    # Close Elasticsearch connection
    es_client.close()
    print("✅ Elasticsearch connection closed")

## 1. Creating the Lambda Function for Elasticsearch Integration

The first step in our journey is to create an AWS Lambda function that will query Elasticsearch. This Lambda function will later be transformed into an MCP tool by AgentCore Gateway, making it accessible to our AI agent.

### The Lambda Function Design

Our Lambda function will:
1. Accept a search query as input
2. Connect to Elasticsearch using stored credentials
3. Perform a multi-match query across document content
4. Return relevant search results to the agent

### Step 1: Create Project Structure

First, we'll create a directory to organize our Lambda function code:

In [ ]:
# Create directory structure for Lambda package
os.makedirs("elastic_lambda_code/package", exist_ok=True)
print("✅ Created Lambda project directories")

### Step 2: Define Lambda Dependencies

Our Lambda function needs the Elasticsearch Python client to query our data store:

In [ ]:
%%writefile elastic_lambda_code/requirements.txt
elasticsearch

### Step 3: Create the Lambda Function Code

Now let's create the actual Lambda function. This function will be invoked by AgentCore Gateway and will query Elasticsearch based on the tool name and parameters passed by the agent.

#### Key Components:

1. **Environment Variables**: The function reads Elasticsearch credentials from environment variables (set during Lambda creation)
2. **Event Schema**: The `event` parameter contains the tool input (e.g., search query)
3. **Context Metadata**: The `context.client_context.custom` contains Gateway metadata including tool name
4. **Tool Name Parsing**: We extract the actual tool name by removing the Gateway target prefix
5. **Elasticsearch Query**: We perform a multi-match search across document content

> 💡 **Gateway Context**: AgentCore Gateway automatically injects metadata into the Lambda context, including the gateway ID, target ID, and tool name. This allows a single Lambda function to implement multiple tools.

In [ ]:
%%writefile elastic_lambda_code/lambda_function_code.py
import json
import boto3
from elasticsearch import Elasticsearch
import os

# Load Elasticsearch configuration from environment variables
endpoint_url = os.environ['ELASTIC_ENDPOINT_URL_ENV']
api_key = os.environ['ELASTIC_API_KEY_ENV']
index_name = os.environ['ELASTIC_INDEX_NAME_ENV']

# Initialize Elasticsearch client
client = Elasticsearch(
    endpoint_url,
    api_key=api_key
)

def lambda_handler(event, context):
    """
    Lambda handler for Elasticsearch search via AgentCore Gateway.
    
    Args:
        event: Contains the tool input matching the inputSchema defined in Gateway
        context: Contains Gateway metadata in context.client_context.custom
    
    Context structure:
        {
            'bedrockAgentCoreGatewayId': 'Y02ERAYBHB',
            'bedrockAgentCoreTargetId': 'RQHDN3J002',
            'bedrockAgentCoreMessageVersion': '1.0',
            'bedrockAgentCoreToolName': 'target_name___elastic_rag_tool',
            'bedrockAgentCoreSessionId': ''
        }
    """
    
    # Extract tool name from Gateway context
    # Gateway prefixes tool names with target name and delimiter
    delimiter = "___"
    original_tool_name = context.client_context.custom['bedrockAgentCoreToolName']
    tool_name = original_tool_name[original_tool_name.index(delimiter) + len(delimiter):]
    
    # Route to appropriate tool handler
    if tool_name == 'elastic_rag_tool':
        # Extract search query from event
        query = event["query"]
        
        # Build Elasticsearch query
        # Using multi_match to search across document content fields
        retriever_object = {
            "standard": {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": [
                            "attachment.content" 
                        ]
                    }
                }
            }
        }
        
        # Execute search
        search_response = client.search(
            index=index_name,
            retriever=retriever_object,
        )
        
        # Return results
        return {'statusCode': 200, 'body': search_response.body}
    else:
        # Handle unsupported tools
        return {'statusCode': 400, 'body': "Unsupported tool."}

### Step 4: Package Lambda Deployment Bundle

Now we'll create the deployment package by:
1. Installing dependencies to the `package/` directory
2. Creating a zip file with all dependencies
3. Adding our Lambda function code to the zip

This process follows AWS Lambda's [deployment package specifications](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html).

In [ ]:
# Store original directory
original_dir = os.getcwd()
project_dir = "elastic_lambda_code"

try:
    print("📦 Building Lambda deployment package...")
    
    # Navigate to project directory
    os.chdir(project_dir)
    print(f"✅ Changed to: {os.getcwd()}")
    
    # Ensure package directory exists
    os.makedirs("package", exist_ok=True)
    print("✅ Created package directory")
    
    # Install dependencies
    print("📥 Installing dependencies...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", 
        "-r", "requirements.txt",
        "--target", "./package",
        "--quiet"
    ])
    print("✅ Installed dependencies to ./package")
    
    # Create zip with dependencies
    os.chdir("package")
    print("🗜️ Creating deployment package...")
    subprocess.check_call([
        "zip", "-r", "../my_deployment_package.zip", ".", "-q"
    ])
    print("✅ Created zip with dependencies")
    
    # Add Lambda function code
    os.chdir("..")
    subprocess.check_call([
        "zip", "my_deployment_package.zip", "lambda_function_code.py", "-q"
    ])
    print("✅ Added lambda_function_code.py to zip")
    
    # Move to parent directory
    shutil.move("my_deployment_package.zip", "../my_deployment_package.zip")
    print("✅ Moved deployment package to project root")
    
finally:
    # Always return to original directory
    os.chdir(original_dir)
    print(f"✅ Returned to: {os.getcwd()}")

print("\n🎉 Lambda deployment package created successfully!")

### Step 5: Deploy the Lambda Function

Now let's create the actual Lambda function in AWS using our deployment package. The utility function will:
- Create an IAM execution role for the Lambda
- Upload the deployment package
- Configure environment variables for Elasticsearch access
- Set appropriate timeout and memory settings

> ⚠️ **Security Note**: The Lambda function will have access to your Elasticsearch instance via the API key stored in environment variables. In production, consider using AWS Secrets Manager for enhanced security.

In [ ]:
print("🚀 Creating Lambda function...")

# Configure Lambda environment variables
environment_variables = {
    'ELASTIC_ENDPOINT_URL_ENV': ELASTIC_ENDPOINT_URL,
    'ELASTIC_API_KEY_ENV': ELASTIC_API_KEY,
    'ELASTIC_INDEX_NAME_ENV': ELASTIC_INDEX_NAME,
}

# Create Lambda function (Can take up to 2 min)
lambda_resp = utils.create_gateway_lambda("my_deployment_package.zip", environment_variables)

# Check creation status
if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print(f"✅ Lambda function created successfully")
        print(f"   ARN: {lambda_resp['lambda_function_arn']}")
        lambda_function_arn = lambda_resp['lambda_function_arn']
    else:
        print(f"❌ Lambda function creation failed: {lambda_resp['lambda_function_arn']}")
        raise Exception("Lambda creation failed")
else:
    print("❌ Lambda function creation returned None")
    raise Exception("Lambda creation failed")

## 2. Configuring Authentication for AgentCore Gateway

Security is paramount when building production AI systems. AgentCore Gateway implements a **dual authentication model** that provides defense-in-depth:

### Understanding Dual Authentication

**1. Inbound Authentication (Who can call the Gateway?)**
- Validates users/agents attempting to access Gateway tools
- Uses OAuth 2.0 tokens from Amazon Cognito
- Ensures only authorized clients can invoke MCP tools

**2. Outbound Authentication (How does Gateway access resources?)**
- Authorizes Gateway to call backend services (Lambda, APIs)
- Uses AWS IAM roles for Lambda invocation
- Enables secure, auditable access to downstream resources

### The Authentication Flow

```
Client (Agent)
    │
    │ [1] Request + OAuth Token
    ↓
AgentCore Gateway
    │
    │ [2] Validate OAuth Token (Inbound Auth)
    │
    │ [3] Assume IAM Role (Outbound Auth)
    ↓
AWS Lambda → Elasticsearch
```

Let's set up both authentication layers.

## 3. Setting Up Amazon Cognito for Inbound Authorization

Amazon Cognito serves as our identity provider (IdP), managing user authentication and issuing OAuth tokens that grant access to our Gateway.

### What We'll Create

1. **Cognito User Pool**: Container for user identities and authentication settings
2. **Resource Server**: Defines custom OAuth scopes for fine-grained access control
3. **App Client**: Machine-to-machine (M2M) client for programmatic access
4. **OAuth Scopes**: `gateway:read` and `gateway:write` permissions

> 💡 **Production Tip**: In production environments, use principle of least privilege—grant only the minimum scopes required for each client's functionality.

In [ ]:
print("🔐 Setting up Amazon Cognito authentication...\n")

# Configuration
USER_POOL_NAME = "agentcore-gateway-elasticsearch-pool"
RESOURCE_SERVER_ID = "elasticsearch-gateway"
RESOURCE_SERVER_NAME = "Elasticsearch Gateway Resource Server"
CLIENT_NAME = "elasticsearch-gateway-client"

# Define OAuth scopes for access control
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access to Gateway tools"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access to Gateway tools"}
]
scope_string = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

# Initialize Cognito client
cognito = boto3.client("cognito-idp", region_name=REGION)

# Step 1: Create or retrieve User Pool
print("📋 Creating/retrieving User Pool...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"✅ User Pool ID: {user_pool_id}")

# Step 2: Create or retrieve Resource Server
print("\n🔧 Configuring Resource Server...")
utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("✅ Resource Server configured with OAuth scopes")

# Step 3: Create or retrieve M2M App Client
print("\n🔑 Creating Machine-to-Machine client...")
client_id, client_secret = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"✅ Client ID: {client_id}")
print("✅ Client Secret generated (stored securely)")

# Step 4: Generate discovery URL for Gateway configuration
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(f"\n🌐 Discovery URL: {cognito_discovery_url}")

print("\n🎉 Cognito authentication setup complete!")

## 4. Creating the AgentCore Gateway

Now we're ready to create our AgentCore Gateway—the central hub that transforms our Lambda function into an MCP-compliant tool accessible by AI agents.

### What is AgentCore Gateway?

AgentCore Gateway is a fully-managed service that:
- **Standardizes Integration**: Provides uniform MCP interfaces across different backend services
- **Manages Security**: Handles both inbound and outbound authentication
- **Eliminates Infrastructure**: No servers to manage or maintain
- **Scales Automatically**: Handles varying loads without configuration

### Gateway Components

Creating a Gateway involves:
1. **IAM Execution Role**: Grants Gateway permission to invoke Lambda functions
2. **Gateway Resource**: The main Gateway instance with authentication configuration
3. **Gateway Target**: Connects the Gateway to your Lambda function

Let's create each component.

### Step 1: Create IAM Execution Role

The Gateway needs an IAM role to invoke Lambda functions on your behalf. This role implements the **outbound authentication** we discussed earlier.

In [ ]:
print("🔐 Creating IAM execution role for Gateway...")

# Create role with appropriate trust policy and permissions
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("elasticsearch-gateway")
gateway_role_arn = agentcore_gateway_iam_role['Role']['Arn']

print(f"✅ Gateway IAM Role created")
print(f"   ARN: {gateway_role_arn}")

### Step 2: Create Gateway with Cognito Authorization

Now we'll create the Gateway itself, configuring it to use our Cognito User Pool for inbound authentication.

#### Configuration Details:

- **Protocol Type**: `MCP` - Model Context Protocol for standardized tool interfaces
- **Authorizer Type**: `CUSTOM_JWT` - Uses Cognito JWT tokens for authentication
- **Allowed Clients**: List of Cognito client IDs permitted to access the Gateway
- **Discovery URL**: Cognito's OIDC discovery endpoint for token validation

> 💡 **Security Note**: Only clients listed in `allowedClients` can successfully invoke Gateway tools, even with a valid JWT token. This provides an additional layer of access control.

In [ ]:
print("🚪 Creating AgentCore Gateway...\n")

# Initialize Gateway client
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

# Configure Cognito JWT authorization
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Must match Cognito Client ID
        "discoveryUrl": cognito_discovery_url
    }
}

# Generate unique Gateway name
gateway_name = f'ElasticsearchGateway-{str(uuid.uuid4())[:8]}'

# Create Gateway
try:
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        protocolType='MCP',
        authorizerType='CUSTOM_JWT',
        authorizerConfiguration=auth_config,
        description='AgentCore Gateway for Elasticsearch integration via Lambda'
    )
    
    # Extract Gateway details
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    
    print(f"✅ Gateway created successfully!")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {gateway_url}")
    
    # Store Gateway ID in Parameter Store for later use
    put_ssm_parameter("/app/customer_support_elastic/agentcore/gateway_id", gateway_id)
    print(f"\n✅ Gateway ID stored in SSM Parameter Store")
    
except ClientError as e:
    print(f"❌ Error creating Gateway: {e}")
    raise

## 5. Creating the Gateway Target and MCP Tools

The final step in setting up our Gateway is creating a **Gateway Target**—this connects our Lambda function to the Gateway and defines how it should be exposed as MCP tools.

### Understanding Gateway Targets

A Gateway Target:
- **Maps Lambda to MCP**: Transforms your Lambda function into one or more MCP tools
- **Defines Tool Schema**: Specifies tool names, descriptions, and input parameters
- **Configures Credentials**: Sets up how Gateway authenticates to Lambda (IAM, API key, OAuth)

### Our Tool Definition

We're creating a single tool called `elastic_rag_tool` that:
- **Name**: `elastic_rag_tool`
- **Purpose**: Performs semantic search on Elasticsearch
- **Input**: A `query` string parameter
- **Output**: Relevant search results from Elasticsearch

### Tool Schema Structure

The tool schema follows JSON Schema format:
```json
{
    "name": "elastic_rag_tool",
    "description": "tool to perform search on Elastic",
    "inputSchema": {
        "type": "object",
        "properties": {
            "query": {"type": "string"}
        },
        "required": ["query"]
    }
}
```

This schema tells the AI agent what parameters the tool expects and which are required.

> 💡 **Multi-Tool Lambda**: A single Lambda function can implement multiple tools by checking the tool name in the context (as we did in our Lambda code).

In [ ]:
print("🔧 Creating Gateway Target for Lambda...\n")

# Define Lambda target configuration with MCP tool schema
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_function_arn,
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "elastic_rag_tool",
                        "description": "Searches Elasticsearch for relevant product policy information. Use this tool when customers ask about product documentation, troubleshooting guides, warranty information, or any product-related questions.",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "query": {
                                    "type": "string",
                                    "description": "The search query to find relevant documents in Elasticsearch"
                                }
                            },
                            "required": ["query"]
                        }
                    }
                ]
            }
        }
    }
}

# Configure credential provider (outbound authentication)
credential_config = [
    {
        "credentialProviderType": "GATEWAY_IAM_ROLE"  # Use Gateway's IAM role for Lambda invocation
    }
]

# Generate unique target name
target_name = f'ElasticSearchTarget-{str(uuid.uuid4())[:8]}'

# Create Gateway Target
try:
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description='Lambda target for Elasticsearch RAG functionality',
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config
    )
    
    target_id = response['targetId']
    
    print(f"✅ Gateway Target created successfully!")
    print(f"   Target ID: {target_id}")
    print(f"   Target Name: {target_name}")
    print(f"\n🎉 Lambda function is now available as MCP tool: {target_name}___elastic_rag_tool")
    
except ClientError as e:
    print(f"❌ Error creating Gateway Target: {e}")
    raise

## 6. Testing the Gateway with a Strands Agent

Now that we've set up our complete infrastructure, let's test it by creating a Strands agent that can access our Elasticsearch data through the Gateway.

### How MCP Integration Works

The Strands agent integrates with AgentCore Gateway using the Model Context Protocol (MCP) specification. Here's the flow:

```
┌─────────────────┐
│  Strands Agent  │
└────────┬────────┘
         │
         │ 1. ListTools (discover available tools)
         ↓
┌─────────────────────┐
│  AgentCore Gateway  │ ← OAuth Token
└─────────┬───────────┘
          │
          │ 2. InvokeTool (call specific tool)
          ↓
┌──────────────────┐
│   AWS Lambda     │
└────────┬─────────┘
         │
         │ 3. Query
         ↓
┌──────────────────┐
│  Elasticsearch   │
└──────────────────┘
```

### MCP Client Configuration

The Strands agent uses an `MCPClient` to communicate with the Gateway:
- **Transport**: HTTP(S) with bearer token authentication
- **Headers**: OAuth token for inbound authentication
- **Protocol**: Standard MCP operations (ListTools, InvokeTools)

Let's start by requesting an access token from Cognito.

### Step 1: Request OAuth Access Token

Before we can call the Gateway, we need to obtain an OAuth access token from Cognito. This token proves our identity and grants us access to the Gateway tools.

> ⏰ **Token Expiration**: Cognito access tokens are typically valid for 1 hour. If your token expires, you'll need to request a new one using the same process.

In [ ]:
print("🔑 Requesting OAuth access token from Cognito...\n")

# Wait for Cognito domain propagation
print("⏳ Waiting for Cognito domain propagation...")
time.sleep(10)

try:
    # Request access token
    token_response = utils.get_token(user_pool_id, client_id, client_secret, scope_string, REGION)
    access_token = token_response["access_token"]
    
    print("✅ Access token obtained successfully")
    print(f"   Token type: Bearer")
    print(f"   Scopes: {scope_string}")
    print(f"   Token (truncated): {access_token[:50]}...")
    
except Exception as e:
    print(f"❌ Error obtaining token: {e}")
    print("\n💡 Tip: If this fails, wait a few minutes for Cognito domain propagation to complete, then try again.")
    raise

### Step 2: Create MCP Client and Agent

Now let's create our Strands agent with MCP client integration. The agent will:
1. Connect to the Gateway using the OAuth token
2. Discover available tools via the `ListTools` MCP operation
3. Load those tools into its configuration
4. Be ready to invoke them based on user queries

> 💡 **Model Choice**: We're using Claude Haiku 4.5, but you can substitute any Bedrock-supported model.

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent
import logging

# Configure logging to see agent reasoning
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logging.getLogger("strands").setLevel(logging.INFO)

print("🤖 Creating Strands agent with MCP tools...\n")

# Create HTTP transport factory for MCP client
def create_streamable_http_transport():
    """Factory function to create authenticated HTTP transport for MCP."""
    return streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"}
    )

# Initialize MCP client
mcp_client = MCPClient(create_streamable_http_transport)

# Create Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=REGION,
)

print("✅ MCP client configured")
print("✅ Bedrock model initialized")

### Step 3: Test the Agent

Let's test our agent with various queries to verify the complete integration:

1. **List available tools** - Verify MCP tool discovery
2. **Search Elasticsearch** - Test the RAG functionality
3. **Answer customer queries** - Validate end-to-end workflow

In [ ]:
print("🧪 Testing agent with MCP tools...\n")
print("=" * 70)

with mcp_client:
    # Step 1: Discover available tools
    print("\n📋 Step 1: Discovering MCP tools...")
    tools = mcp_client.list_tools_sync()
    print(f"✅ Found {len(tools)} tool(s)")
    
    # Step 2: Create agent with tools
    print("\n🤖 Step 2: Creating agent...")
    agent = Agent(model=model, tools=tools)
    print(f"✅ Agent created with tools: {agent.tool_names}")
    
    # Step 3: Test tool listing query
    print("\n" + "=" * 70)
    print("Test 1: List available tools")
    print("=" * 70)
    response = agent("Hi, can you list all tools available to you?")
    print(f"\n🤖 Agent: {response.message['content'][0]['text']}")
    
    # Step 4: Test Elasticsearch search
    print("\n" + "=" * 70)
    print("Test 2: Search Elasticsearch for product information")
    print("=" * 70)
    response = agent("Use the elastic_rag_tool to find information about how to troubleshoot the LAPTOP ULTRA 15.6")
    print(f"\n🤖 Agent: {response.message['content'][0]['text']}")

print("\n" + "=" * 70)
print("✅ All tests completed successfully!")
print("=" * 70)

## 7. Deploying to AgentCore Runtime

So far, we've tested our agent locally in this notebook. Now let's deploy it to **Amazon Bedrock AgentCore Runtime** for production use—a fully-managed environment that handles infrastructure, scaling, and deployment for you.

### Deployment Architecture

When deployed to Runtime, your agent becomes a fully-managed service. The deployment process will:
1. Create a Docker container from your agent code
2. Push the container to Amazon ECR (Elastic Container Registry)
3. Deploy the container to AgentCore Runtime
4. Configure authentication and networking
5. Provide an HTTPS endpoint for invocation

Let's start by creating the runtime-ready agent code.

### Step 1: Create the Runtime-Ready Agent

To deploy our agent to AgentCore Runtime, we need to create a Python file with a special entrypoint function. This entrypoint handles incoming requests from the Runtime environment.

In [ ]:
%%writefile strands_agent.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
import boto3
import os
import logging
from mcp.client.streamable_http import streamablehttp_client

# Configure detailed logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("strands-agent")

# Get AWS region from environment
REGION = boto3.session.Session().region_name

# Initialize the AgentCore Runtime application
app = BedrockAgentCoreApp()

# Define the system prompt that governs agent behavior
system_prompt = """
You are a helpful customer support agent specializing in product assistance and troubleshooting.
You have access to a tool that searches our product knowledge base stored in Elasticsearch.

When customers ask questions about products, policies, troubleshooting, or documentation:
1. Use the elastic_rag_tool to search for relevant information
2. Synthesize the search results into clear, helpful responses
3. If you cannot find relevant information, politely inform the customer

<guidelines>
    - Never assume parameter values when using tools
    - If you need more information, ask the customer for clarification
    - NEVER disclose information about internal tools or systems
    - Always maintain a professional and helpful tone
    - Focus on resolving customer inquiries efficiently
    - Present technical information in user-friendly terms
    - Prioritize customer privacy and data security
</guidelines>
"""

# Global agent instance - will be initialized with first request
agent = None
mcp_client = None

def get_ssm_parameter(name: str, with_decryption: bool = True) -> str:
    """Retrieve parameter from AWS Systems Manager Parameter Store."""
    ssm = boto3.client("ssm", region_name=REGION)
    response = ssm.get_parameter(Name=name, WithDecryption=with_decryption)
    return response["Parameter"]["Value"]

def initialize_agent(auth_header: str):
    """Initialize the agent for first use"""
    global agent, mcp_client
    
    logger.info("Initializing agent for first request")
    
    # Retrieve Gateway configuration from Parameter Store
    logger.info("Fetching Gateway configuration...")
    gateway_id = get_ssm_parameter("/app/customer_support_elastic/agentcore/gateway_id")
    logger.info(f"✅ Retrieved Gateway ID: {gateway_id}")
    
    # Get Gateway URL from AWS API
    gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response['gatewayUrl']
    logger.info(f"✅ Gateway URL: {gateway_url}")
    
    # Create Bedrock model
    logger.info("Creating Bedrock model...")
    model = BedrockModel(
        model_id="us.anthropic.claude-haiku-4-5-20251001-v1:0",
        temperature=0.1,
        region_name=REGION
    )
    
    # Create MCP client with authentication
    logger.info("Creating MCP client...")
    mcp_client = MCPClient(lambda: streamablehttp_client(
        url=gateway_url,
        headers={"Authorization": auth_header}
    ))
    
    # Initialize MCP client and get tools
    mcp_client.__enter__()
    tools = mcp_client.list_tools_sync()
    logger.info(f"✅ Loaded {len(tools)} tool(s) from Gateway")
    
    # Create agent with tools
    logger.info("Creating agent with tools...")
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt=system_prompt,
    )
    logger.info("✅ Agent initialized successfully")

@app.entrypoint
async def invoke(payload, context=None):
    """
    AgentCore Runtime entrypoint function.
    
    Args:
        payload: Dictionary containing 'prompt' key with user input
        context: Runtime context with request_headers containing Authorization token
    
    Returns:
        str: Agent response text
    """
    global agent
    
    # Log payload and context
    logger.info(f"Received payload: {payload}")
    logger.info(f"Context: {context}")
    
    # Extract user input
    user_input = payload.get("prompt")
    
    # Validate required fields
    if user_input is None:
        error_msg = "❌ ERROR: Missing 'prompt' field in payload"
        logger.error(error_msg)
        return error_msg
    
    # Get request headers (handle None case)
    request_headers = context.request_headers or {}
    
    # Extract JWT token for Gateway authentication
    auth_header = request_headers.get('Authorization', '')
    
    if not auth_header:
        return "Error: Missing Authorization header. Please provide a valid OAuth token."
    
    logger.info(f"✅ Authorization header present: {auth_header[:20]}...")
    
    try:
        # Initialize agent on first request
        if agent is None:
            logger.info("First request - initializing agent")
            initialize_agent(auth_header)
        else:
            logger.info("Using existing agent instance")
        
        # Invoke agent with user input
        logger.info(f"🤖 Processing query: {user_input[:50]}...")
        response = agent(user_input)
        
        # Extract and return response text
        response_text = response.message["content"][0]["text"]
        logger.info(f"✅ Agent response: {response_text[:50]}...")
        
        return response_text
        
    except Exception as e:
        error_msg = f"Error processing request: {str(e)}"
        logger.error(f"❌ {error_msg}", exc_info=True)
        return error_msg

if __name__ == "__main__":
    # Start the HTTP server
    logger.info("Starting AgentCore application")
    app.run()

### Step 2: Configure Runtime Deployment

Now we'll configure the deployment using the **AgentCore Starter Toolkit**.

> 💡 **Header Allowlist**: We configure the Runtime to forward the `Authorization` header to our agent code, enabling OAuth token propagation through the entire stack.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from utils_execution import create_agentcore_runtime_execution_role

print("⚙️ Configuring AgentCore Runtime deployment...\n")

# Create execution role for Runtime
print("🔐 Creating execution role...")
execution_role_arn = create_agentcore_runtime_execution_role()
print(f"✅ Execution role created: {execution_role_arn}")

# Initialize Runtime toolkit
agentcore_runtime = Runtime()

# Configure deployment
print("\n📋 Configuring deployment parameters...")
response = agentcore_runtime.configure(
    entrypoint="strands_agent.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,  # Automatically create ECR repository
    requirements_file="requirements.txt",
    region=REGION,
    agent_name="customer_support_elasticsearch_agent",
    non_interactive=True,
    memory_mode="NO_MEMORY",  # We're not using AgentCore Memory in this example
    # Configure Cognito JWT authorization
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [client_id],
            "discoveryUrl": cognito_discovery_url
        }
    },
    # Allow Authorization header to be forwarded to agent
    request_header_configuration={
        "requestHeaderAllowlist": [
            "Authorization"  # Required for OAuth token propagation
        ]
    },
)

print("\n✅ Configuration completed successfully")
print("\n📄 Generated files:")
print("   - Dockerfile")
print("   - .bedrock_agentcore.yaml")

### Step 3: Launch the Agent to Runtime

Now comes the exciting part—launching your agent to production! This step will:

1. **Build Docker Image**: Creates a container with your agent code and dependencies
2. **Push to ECR**: Uploads the image to Amazon Elastic Container Registry
3. **Create Runtime**: Provisions the AgentCore Runtime infrastructure
4. **Configure Networking**: Sets up secure HTTPS endpoints
5. **Deploy Container**: Runs your agent in the managed environment

> ⏱️ **Deployment Time**: Initial deployment typically can take up to 10 minutes. The toolkit will create an AWS CodeBuild project to build your container.

> 💡 **Update Existing Runtime**: If an agent with the same name already exists, use `auto_update_on_conflict=True` to update it instead of creating a new one.

In [ ]:
print("🚀 Launching agent to AgentCore Runtime...\n")
print("This process will:")
print("  1. Build Docker container")
print("  2. Push to Amazon ECR")
print("  3. Deploy to AgentCore Runtime")
print("  4. Configure authentication and networking\n")
print("⏳ This may take several minutes...\n")

try:
    # Launch the agent
    launch_result = agentcore_runtime.launch()
    
    print("\n✅ Launch initiated successfully!")
    print(f"   Agent ARN: {launch_result.agent_arn}")
    print(f"   Agent ID: {launch_result.agent_id}")
    print(f"   ECR URI: {launch_result.ecr_uri}")
    
except Exception as e:
    print(f"\n❌ Launch failed: {e}")
    print("\n💡 Tip: If the agent name already exists, you can update it:")
    print("   launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)")
    raise

## 8. Testing the deployment

Congratulations! Your agent is now deployed and running. Let's test it with various customer support scenarios.

### Testing Strategy

We'll test three key scenarios:
1. **Product Troubleshooting**: Testing Elasticsearch retrieval with specific product queries
2. **General Information**: Testing how the agent handles queries requiring knowledge base search
3. **Session Continuity**: Verifying the agent session handling

### Authentication for Testing

First, we need a fresh OAuth token (tokens expire after a period of time):

In [ ]:
print("🔑 Obtaining fresh OAuth token for testing...\n")

try:
    token_response = utils.get_token(user_pool_id, client_id, client_secret, scope_string, REGION)
    test_token = token_response["access_token"]
    
    print("✅ Token obtained successfully")
    print(f"   Token (truncated): {test_token[:50]}...")
except Exception as e:
    print(f"❌ Error obtaining token: {e}")
    raise

### Test Scenario 1: Product Troubleshooting

Let's test our agent with a customer asking about troubleshooting a specific product:

In [ ]:
print("🧪 Test 1: Product Troubleshooting\n")
print("=" * 70)

user_query = "How can I troubleshoot my LAPTOP ULTRA 15.6?"
print(f"👤 Customer: {user_query}\n")

try:
    response = agentcore_runtime.invoke(
        {"prompt": user_query},
        bearer_token=test_token,
    )
    
    print(f"🤖 Agent Response:")
    print("-" * 70)
    print(response["response"])
    print("=" * 70)
    print("\n✅ Test 1 completed successfully\n")
    
except Exception as e:
    print(f"❌ Test failed: {e}\n")

### Test Scenario 2: General Product Information

Let's test with a different type of query about product support:

In [ ]:
print("🧪 Test 2: General Product Information\n")
print("=" * 70)

user_query = "What number can I call for health related questions on my Smart Watch Series X?"
print(f"👤 Customer: {user_query}\n")

try:
    response = agentcore_runtime.invoke(
        {"prompt": user_query},
        bearer_token=test_token,
    )
    
    print(f"🤖 Agent Response:")
    print("-" * 70)
    print(response["response"])
    print("=" * 70)
    print("\n✅ Test 2 completed successfully\n")
    
except Exception as e:
    print(f"❌ Test failed: {e}\n")

### Test Scenario 3: Session Management

Let's test session continuity by asking a follow-up question:

In [ ]:
print("🧪 Test 3: Session Management\n")
print("=" * 70)

# Create a session ID for continuity
session_id = f"test-session-{uuid.uuid4()}"

# First query
user_query_1 = "What information do you have for Smart Watch Series X?"
print(f"👤 Customer (Message 1): {user_query_1}\n")

response_1 = agentcore_runtime.invoke(
    {"prompt": user_query_1},
    bearer_token=test_token,
    session_id=session_id
)

print(f"🤖 Agent: {response_1['response'][:200]}...\n")

# Follow-up query
user_query_2 = "Can you tell me more about the specific product I asked about?"
print(f"👤 Customer (Message 2): {user_query_2}\n")

response_2 = agentcore_runtime.invoke(
    {"prompt": user_query_2},
    bearer_token=test_token,
    session_id=session_id
)

print(f"🤖 Agent:")
print("-" * 70)
print(response_2['response'])
print("=" * 70)
print("\n✅ Test 3 completed successfully\n")

## Key Concepts Summary

Congratulations on completing this tutorial! Let's review the key concepts you've learned:

### 1. AgentCore Gateway Architecture
- **MCP Transformation**: How to transform Lambda functions into Model Context Protocol (MCP) tools
- **Dual Authentication**: Implementing both inbound (OAuth) and outbound (IAM) authentication
- **Managed Infrastructure**: Leveraging fully-managed Gateway services to eliminate operational overhead

### 2. Authentication & Security
- **Cognito Integration**: Setting up OAuth 2.0 authentication with Amazon Cognito
- **Token Propagation**: Forwarding JWT tokens through Runtime to Gateway
- **IAM Roles**: Configuring least-privilege access for Gateway and Runtime
- **Defense in Depth**: Multiple layers of security from client to data store

### 3. Elasticsearch Integration
- **RAG Pattern**: Implementing Retrieval-Augmented Generation with Elasticsearch
- **Lambda as Data Bridge**: Using Lambda to securely connect Gateway to Elasticsearch
- **Semantic Search**: Performing multi-match queries for relevant document retrieval

### 4. AgentCore Runtime Deployment
- **Containerization**: Packaging agents as Docker containers for portability
- **Starter Toolkit**: Using automation tools to simplify deployment processes
- **Production Readiness**: Deploying scalable, managed agent services
- **Header Configuration**: Forwarding authentication and custom headers to agents

### 5. MCP Protocol
- **Standard Interface**: Using MCP for consistent tool integration
- **Tool Discovery**: Dynamic tool loading via `ListTools` operation
- **Tool Invocation**: Executing tools via `InvokeTools` operation
- **Client Libraries**: Leveraging MCP client SDKs for seamless integration

### Architecture Benefits

This architecture provides several production advantages:

✅ **Scalability**: All components (Gateway, Runtime, Lambda) scale automatically

✅ **Security**: Multiple authentication layers protect every access point

✅ **Maintainability**: Separation of concerns makes updates and debugging easier

✅ **Flexibility**: Easy to add new tools, data sources, or agents

✅ **Cost Efficiency**: Pay only for actual usage with serverless components

## Cleanup (Optional)

To avoid incurring unnecessary AWS charges, you can delete the resources created in this tutorial. This section will guide you through cleaning up:

1. **AgentCore Runtime Agent** - The deployed agent container
2. **AgentCore Gateway** - The MCP Gateway and targets
3. **Lambda Function** - The Elasticsearch query function
4. **Cognito Resources** - User pool and app clients
5. **IAM Roles** - Execution roles created for services
6. **ECR Repository** - Container image repository
7. **SSM Parameters** - Stored configuration values

> ⚠️ **Warning**: This action is irreversible. Make sure you want to delete these resources before proceeding.

### Cleanup Steps

In [ ]:
# ⚠️ WARNING: Only run this cell if you want to delete all created resources

print("🧹 Starting cleanup process...\n")
print("=" * 70)

cleanup_errors = []

# 1. Delete AgentCore Runtime
print("\n1️⃣ Deleting AgentCore Runtime...")
try:
    if 'launch_result' in locals() and hasattr(launch_result, 'agent_id'):
        runtime_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
        runtime_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
        print(f"   ✅ Deleted Runtime: {launch_result.agent_id}")
    else:
        print("   ⏭️  No Runtime to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"Runtime deletion: {e}")

# 2. Delete Gateway Target
print("\n2️⃣ Deleting Gateway Target...")
try:
    if 'target_id' in locals() and 'gateway_id' in locals():
        gateway_client.delete_gateway_target(
            gatewayIdentifier=gateway_id,
            targetId=target_id
        )
        print(f"   ✅ Deleted Target: {target_id}")
    else:
        print("   ⏭️  No Gateway Target to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"Gateway Target deletion: {e}")

# 3. Delete Gateway
time.sleep(10)
print("\n3️⃣ Deleting AgentCore Gateway...")
try:
    if 'gateway_id' in locals():
        gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
        print(f"   ✅ Deleted Gateway: {gateway_id}")
    else:
        print("   ⏭️  No Gateway to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"Gateway deletion: {e}")

# 4. Delete Lambda Function
print("\n4️⃣ Deleting Lambda Function...")
try:
    if 'lambda_function_arn' in locals():
        lambda_client = boto3.client('lambda', region_name=REGION)
        function_name = lambda_function_arn.split(':')[-1]
        lambda_client.delete_function(FunctionName=function_name)
        print(f"   ✅ Deleted Lambda: {function_name}")
    else:
        print("   ⏭️  No Lambda function to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"Lambda deletion: {e}")

# 5. Delete Cognito User Pool
print("\n5️⃣ Deleting Cognito User Pool…")
try:
    if 'user_pool_id' in locals():
        # First, delete the domain if it exists
        try:
            response = cognito.describe_user_pool(UserPoolId=user_pool_id)
            domain = response.get('UserPool', {}).get('Domain')
            if domain:
                cognito.delete_user_pool_domain(
                    Domain=domain,
                    UserPoolId=user_pool_id
                )
                print(f"   ✅ Deleted User Pool Domain: {domain}")
        except cognito.exceptions.ResourceNotFoundException:
            print("   ⏭️  No domain found")
        except Exception as domain_error:
            print(f"   ⚠️  Domain deletion warning: {domain_error}")
        
        # Now delete the user pool
        cognito.delete_user_pool(UserPoolId=user_pool_id)
        print(f"   ✅ Deleted User Pool: {user_pool_id}")
    else:
        print("   ⏭️  No User Pool to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"Cognito deletion: {e}")

# 6. Delete ECR Repository
print("\n6️⃣ Deleting ECR Repository...")
try:
    if 'launch_result' in locals() and hasattr(launch_result, 'ecr_uri'):
        ecr_client = boto3.client('ecr', region_name=REGION)
        repo_name = launch_result.ecr_uri.split('/')[1].split(':')[0]
        ecr_client.delete_repository(repositoryName=repo_name, force=True)
        print(f"   ✅ Deleted ECR Repository: {repo_name}")
    else:
        print("   ⏭️  No ECR Repository to delete")
except Exception as e:
    print(f"   ❌ Error: {e}")
    cleanup_errors.append(f"ECR deletion: {e}")

# 7. Delete SSM Parameters
print("\n7️⃣ Deleting SSM Parameters...")
try:
    ssm_client = boto3.client('ssm', region_name=REGION)
    ssm_client.delete_parameter(Name="/app/customer_support_elastic/agentcore/gateway_id")
    print("   ✅ Deleted SSM Parameters")
except Exception as e:
    print(f"   ⏭️  No SSM Parameters to delete (or already deleted)")

# Summary
print("\n" + "=" * 70)
if cleanup_errors:
    print("⚠️  Cleanup completed with errors:")
    for error in cleanup_errors:
        print(f"   - {error}")
else:
    print("✅ Cleanup completed successfully!")
print("=" * 70)

print("\n💡 Note: IAM roles may have a deletion delay due to AWS IAM eventual consistency.")
print("   You can manually delete them from the IAM console if needed.")